# Explorasi VinBigData Chest X-ray Abnormaliti dalam Bahasa Indonesia
#### oleh Muhammad Zharfan Wiranata

Pada kompetisi ini, kita akan lokalisasi dan mengklasifikasi 14 tipe thoracic abnormaliti dari radiograf pundak secara otomatis. dengan 18,000 dataset yang telah discan oleh radiolog bepengalaman, kita dapat 15,000 data train yabg berlebel dan independent dan akan dievaluasi dengan sisa 3,000 gambar yang lain. Anotasi ini dikumpulkan oleh VinBigData via webnya VinLab. Detai pada dataset bisa dilihati di [“VinDr-CXR: An open dataset of chest X-rays with radiologist's annotations”.](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/overview)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import os
dataset_folder="/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/"

import matplotlib.pyplot as plt
import pydicom
from matplotlib import patches
file_train = dataset_folder+"train.csv"
df_train = pd.read_csv(file_train)

In [ ]:
# df_train.loc[df_train['image_id'] == "e31be972e181987a8600a8700c1ebe88"] 

## function gambar Review
def GetRawDicom(image_id=""):
    dataset_train_dir = dataset_folder + "train/"
    try:
        filename = dataset_train_dir+image_id+".dicom"
        ds = pydicom.dcmread(filename)
        return ds
    except:
        print("Something Error")
        return False
def ShowRawImage(image_id="",cmap=plt.cm.gray):
    if GetRawDicom(image_id):
        raw_dicom=GetRawDicom(image_id)
        plt.imshow(raw_dicom.pixel_array,cmap=cmap) 
        plt.show()
def ShowImageWithDetectedMark(image_id=""):
    if GetRawDicom(image_id):
        raw_dicom=GetRawDicom(image_id)
        fig, ax = plt.subplots()
        ax.imshow(raw_dicom.pixel_array,cmap=plt.cm.gray)
        abn_det = df_train.loc[df_train['image_id'] == image_id]
        for det in abn_det.iterrows():
            ax.add_patch(
             patches.Rectangle(
                (det[1].x_min,det[1].y_min),
                (det[1].x_max-det[1].x_min),
                (det[1].y_max-det[1].y_min),
                edgecolor = 'red',
                facecolor = 'Blue',
                fill=False     
             ) )
            ax.text(det[1].x_min,det[1].y_min,det[1].class_name,bbox=dict(facecolor='red', alpha=0.1))
        plt.show()
def ZoomShowImageWithDetectedMark(image_id="",x_min=0,x_max=100,y_min=0,y_max=100):
    if GetRawDicom(image_id):
        fig, ax = plt.subplots()
        raw_dicom=GetRawDicom(image_id)
        ax.imshow(raw_dicom.pixel_array,cmap=plt.cm.gray)
        ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))
        abn_det = df_train.loc[df_train['image_id'] == image_id]
        for det in abn_det.iterrows():
            rule1= (det[1].x_min > x_min) and (det[1].x_min < x_max)
            rule2= (det[1].x_max > x_min) and (det[1].x_max < x_max)
            rule3= (det[1].y_min > y_min) and (det[1].y_min < y_max)
            rule4= (det[1].y_max > y_min) and (det[1].y_max < y_max)
            ax.add_patch(
             patches.Rectangle(
                (det[1].x_min,det[1].y_min),
                (det[1].x_max-det[1].x_min),
                (det[1].y_max-det[1].y_min),
                edgecolor = 'red',
                facecolor = 'Blue',
                fill=False     
             ) )
            if (rule1 and rule2 and rule3 and rule4):
                ax.text(det[1].x_min,det[1].y_min,det[1].class_name,bbox=dict(facecolor='red', alpha=0.1))
        plt.show()

<h2 align="center" style="background-color:#005595;color:white;">Eksplorasi File</h2>

In [ ]:
print('Isi pada folder Dataset adalah :')
for i in os.listdir(dataset_folder):
    print(i)
# os.listdir(dataset_folder)

### Tentang folder "test" dan "train"

In [ ]:
dataset_train_dir = dataset_folder + "train"
dataset_test_dir = dataset_folder + "test"
jumlah_filetrain = len(os.listdir(dataset_train_dir))
jumlah_filetest = len(os.listdir(dataset_test_dir))
persentase_train = (jumlah_filetrain/(jumlah_filetrain+jumlah_filetest))*100
persentase_test = (jumlah_filetest/(jumlah_filetrain+jumlah_filetest))*100
print("Jumlah File Train adalah " + str(jumlah_filetrain) + " ("+ str(persentase_train) +"%)")
print("Jumlah File Test adalah " + str(jumlah_filetest) + " ("+ str(persentase_test) +"%)")

### Cek Sample Data Dicom

In [ ]:
data_sample = GetRawDicom("fa109c087e46fe1ea27e48ce6d154d2f")
# print(data_sample.file_meta)
count = 1
for i in data_sample:
    print("data "+str(count)+" :"+str(i))
    count = count+1
count = 1
for i in data_sample.file_meta:
    print("data Meta "+str(count)+" :"+str(i))
    count = count+1

### Tentang sample_submision.csv

In [ ]:
file_sampleSubmission = dataset_folder+"sample_submission.csv"
sample_submission = pd.read_csv(file_sampleSubmission)
sample_submission

### Tentang train.csv

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df_train, title="Train Profiling Report", correlations=None,interactions=None,progress_bar=False)
profile.to_notebook_iframe()

In [ ]:
print("Daftar Abnormaliti nya adalah : ")
for ab in df_train.class_name.unique():
    print(ab)

Kita Pastikan bahwa data abnormaliti sesuai dengan deskripsi data di [“VinDr-CXR: An open dataset of chest X-rays with radiologist's annotations”.](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/overview)
* 0 - Aortic enlargement
* 1 - Atelectasis
* 2 - Calcification
* 3 - Cardiomegaly
* 4 - Consolidation
* 5 - ILD
* 6 - Infiltration
* 7 - Lung Opacity
* 8 - Nodule/Mass
* 9 - Other lesion
* 10 - Pleural effusion
* 11 - Pleural thickening
* 12 - Pneumothorax
* 13 - Pulmonary fibrosis

untuk "No Finding", diberikan "class_id" bernilai 14

#### Ringkasan Profiling
* Perlu diperhatikan, bahwa hampir setengahnya (46.9%) data tidak terdeteksi Abnormaliti.
* Id Gambar **<i style="color:red;">03e6ecfa6f6fb33dfeac6ca4f9b459c9</i>** memiliki repetisi terbanyak pada data train. Yang bisa diartikan banyak terdeteksi <b><u>abnormaliti</u></b>
* 

<h2 align="center" style="background-color:#005595;color:white;">Eksplorasi sampel gambar medis</h2>

### Penampilan Gambar Sampel yang tidak terdeteksi

In [ ]:
# sample_undetected = df_train.loc[df_train['class_id'] == 14]
# sample_undetected

In [ ]:
ShowRawImage("50a418190bc3fb1ef1633bf9678929b3")
ShowRawImage("21a10246a5ec7af151081d0cd6d65dc9")
ShowRawImage("063319de25ce7edb9b1c6b8881290140")

### Penampilan Gambar Sampel mentah

In [ ]:
ShowRawImage("03e6ecfa6f6fb33dfeac6ca4f9b459c9")

### Penampilan Gambar Sampel Bertanda

In [ ]:
ShowImageWithDetectedMark("03e6ecfa6f6fb33dfeac6ca4f9b459c9")
ShowImageWithDetectedMark("c699f16ba0b86f474390da9515bcad7a")
ShowImageWithDetectedMark("dfd523a5991fc852654bf1235c6282c6")

In [ ]:
ZoomShowImageWithDetectedMark("03e6ecfa6f6fb33dfeac6ca4f9b459c9",x_min=1750,x_max=2500,y_min=750,y_max=1250)
ZoomShowImageWithDetectedMark("03e6ecfa6f6fb33dfeac6ca4f9b459c9",x_min=1800,x_max=2000,y_min=2000,y_max=2500)